In [13]:
import logging

import numpy as np
import matplotlib.pyplot as plt

try:
    import keysightSD1
except:
    import sys
    sys.path.append(r"C:\Program Files (x86)\Keysight\SD1\Libraries\Python")
    import keysightSD1
    
import qcodes
from qcodes_contrib_drivers.drivers.Keysight.M3202A import M3202A

import qcodes.logger as logger
from qcodes.logger import start_all_logging

start_all_logging()
# logger.get_file_handler().setLevel(logging.DEBUG)

# try to close station from previous run.
try:
    station.close_all_registered_instruments()
except: pass

# setup station
awg = M3202A("AWG", chassis = 1, slot = 4)

station = qcodes.Station()
station.add_component(awg)

# setup output channels
pxi = keysightSD1.SD_TriggerExternalSources.TRIGGER_PXI1
trigger_mode = keysightSD1.SD_TriggerBehaviors.TRIGGER_FALL

for ch in range(1,5):
    awg.set_channel_offset(0.0, ch)
    awg.set_channel_amplitude(0.5, ch)

    awg.set_channel_wave_shape(keysightSD1.SD_Waveshapes.AOU_AWG, ch)
    awg.awg_queue_config(ch, keysightSD1.SD_QueueMode.CYCLIC)
    awg.awg_config_external_trigger(ch, pxi, trigger_mode)

Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\noah\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


2021-05-21 13:56:18,843 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [AWG(M3202A)] Snapshot: Could not update parameter: status


Qcodes Logfile : C:\Users\noah\.qcodes\logs\210521-20828-qcodes.log


In [14]:
# setup waveforms
from waveforms import create_sine, get_prescaler

sample_rate_MSps = 1000 # 1 GS/s

sine = 0.5 * create_sine(500, 8, sample_rate_MSps)
sine_awg = awg.upload_waveform(sine)

delay = 0
ext_trigger = keysightSD1.SD_TriggerModes.EXTTRIG
auto_trigger = keysightSD1.SD_TriggerModes.AUTOTRIG
cycles = 0
prescaler = get_prescaler(sample_rate_MSps)
awg.awg_queue_waveform(1, sine_awg, auto_trigger, delay, cycles, prescaler)

awg.awg_start(1)

2021-05-21 13:56:22,854 ¦ root ¦ WARNING ¦ SD_AWG_Async ¦ __del__ ¦ 152 ¦ WaveformReference was not released (AWG:0). Automatic release in destructor.


In [ ]:
# trigger AWG

# awg.set_pxi_trigger(0, pxi) # trigger ON
# awg.set_pxi_trigger(1, pxi) # trigger OFF